# **📊 1. 관심 데이터 선정**

> Smilegate AI에서 underscore와 협업하여 만든 데이터셋인 Korean UnSmile Dataset을 사용하였습니다.
(https://github.com/smilegate-ai/korean_unsmile_dataset)






# **❓ 2. 데이터 선정 이유**

*   온라인 환경에서 익명성과 표현의 자유 뒤에 숨어 악의적인 발언으로 상대를 공격이 이전에 주된 악성 댓글이었다면 현재는 불특정 다수인 특정 집단에 대한 혐오 표현 사용까지 진행되는 중입니다.
*   전자의 경우 피해자에겐 큰 정신적 고통이, 후자의 경우 분별없는 수용와 전파로 인한 사회적 인식 변화로 인해 집단 전체가 피해를 봅니다.


> 현재 사용되는 혐오표현들은 의미없는 갈등과 혐오의 재생산을 만들어내 사회적으로 큰 악영향을 끼치는데 이는 포털사이트, 게임, 영상제작 등 다양한 분야에서의 컨텐츠나 서비스에서도 문제가 되는 것을 쉽게 확인할 수 있습니다.

# **🧩 3. 데이터를 이용한 가설 수립**

> 악성 댓글 / 혐오 표현을 사전에 검출, 차단함으로써, 사회적 문제에 대한 기업의 책임 문제 해결과 서비스의 품질 향상을 기대해 볼 수 있습니다.



# **🧹 4. 데이터 전처리**



> 1번의 깃헙 링크에 들어가시면 데이터에 대한 많은 정보를 보실 수 있습니다.



In [ ]:
!pip install transformers
!pip install datasets==1.17.0

In [ ]:
# 데이터 가져오기
from datasets import load_dataset
dataset = load_dataset('smilegate-ai/kor_unsmile')

In [2]:
dataset['train'][0]

{'clean': 1,
 'labels': [0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
 '개인지칭': 0,
 '기타 혐오': 0,
 '남성': 0,
 '문장': '일안하는 시간은 쉬고싶어서 그런게 아닐까',
 '성소수자': 0,
 '악플/욕설': 0,
 '여성/가족': 0,
 '연령': 0,
 '인종/국적': 0,
 '종교': 0,
 '지역': 0}

In [3]:
unsmile_labels = ["여성/가족","남성","성소수자","인종/국적","연령","지역","종교","기타 혐오","악플/욕설","clean"]

In [4]:
dataset.shape

{'train': (15005, 13), 'valid': (3737, 13)}

# **🧠 5. 딥러닝 방식 적용**

In [ ]:
!pip install ipywidgets  # for vscode
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

In [ ]:
!pip install --upgrade urllib3==1.26.7
!pip install --upgrade awscli==1.22.26
!pip install --upgrade botocore==1.23.26

In [ ]:
!pip install seqeval
!pip install fastprogress
!pip install attrdict

In [5]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
from tqdm.notebook import tqdm
from kobert import get_tokenizer
from kobert import get_pytorch_kobert_model
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
import numpy as np
from transformers import BertForSequenceClassification, TrainingArguments, Trainer, AutoTokenizer

In [ ]:
model_name = 'beomi/kcbert-base'
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [7]:
def preprocess_function(examples):
    tokenized_examples = tokenizer(str(examples["문장"]))
    tokenized_examples['labels'] = torch.tensor(examples["labels"], dtype=torch.float)
    # multi label classification 학습을 위해선 label이 float 형태로 변형되어야 합니다.
    # huggingface datasets 최신 버전에는 'map' 함수에 버그가 있어서 변형이 올바르게 되지 않습니다.
    
    return tokenized_examples

In [ ]:
tokenized_dataset = dataset.map(preprocess_function)
tokenized_dataset.set_format(type='torch', columns=['input_ids', 'labels', 'attention_mask', 'token_type_ids'])

In [9]:
tokenized_dataset['train'][0]

{'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1]),
 'input_ids': tensor([    2,  2458, 15751, 24930, 24351, 29278, 17038, 11631,     3]),
 'labels': tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]),
 'token_type_ids': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0])}

In [10]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
num_labels=len(unsmile_labels) # Label 갯수

model = BertForSequenceClassification.from_pretrained(
    model_name, 
    num_labels=num_labels, 
    problem_type="multi_label_classification"
)
model.config.id2label = {i: label for i, label in zip(range(num_labels), unsmile_labels)}
model.config.label2id = {label: i for i, label in zip(range(num_labels), unsmile_labels)}

In [12]:
model.config.label2id

{'clean': 9,
 '기타 혐오': 7,
 '남성': 1,
 '성소수자': 2,
 '악플/욕설': 8,
 '여성/가족': 0,
 '연령': 4,
 '인종/국적': 3,
 '종교': 6,
 '지역': 5}

In [13]:
from sklearn.metrics import label_ranking_average_precision_score

In [14]:
def compute_metrics(x):
    return {
        'lrap': label_ranking_average_precision_score(x.label_ids, x.predictions),
    }

In [15]:
batch_size = 32

In [16]:
args = TrainingArguments(
    output_dir="model_output",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=2,
    save_strategy='epoch',
    load_best_model_at_end=True,
    metric_for_best_model='lrap',
    greater_is_better=True,
)

trainer = Trainer(
    model=model, 
    args=args, 
    train_dataset=tokenized_dataset["train"], 
    eval_dataset=tokenized_dataset["valid"], 
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
    data_collator=data_collator
)

In [17]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: 여성/가족, 남성, 지역, 종교, 개인지칭, clean, 악플/욕설, 연령, 문장, 인종/국적, 성소수자, 기타 혐오. If 여성/가족, 남성, 지역, 종교, 개인지칭, clean, 악플/욕설, 연령, 문장, 인종/국적, 성소수자, 기타 혐오 are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 15005
  Num Epochs = 2
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 938


Epoch,Training Loss,Validation Loss,Lrap
1,No log,0.136910,0.869759
2,0.194800,0.125699,0.878762


The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: 여성/가족, 남성, 지역, 종교, 개인지칭, clean, 악플/욕설, 연령, 문장, 인종/국적, 성소수자, 기타 혐오. If 여성/가족, 남성, 지역, 종교, 개인지칭, clean, 악플/욕설, 연령, 문장, 인종/국적, 성소수자, 기타 혐오 are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 3737
  Batch size = 32
Saving model checkpoint to model_output/checkpoint-469
Configuration saved in model_output/checkpoint-469/config.json
Model weights saved in model_output/checkpoint-469/pytorch_model.bin
tokenizer config file saved in model_output/checkpoint-469/tokenizer_config.json
Special tokens file saved in model_output/checkpoint-469/special_tokens_map.json
The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: 여성/가족, 남성, 지역, 종교, 개인지칭, clean, 악플/욕설, 연령, 문장, 인종/국적, 

TrainOutput(global_step=938, training_loss=0.15797887529645646, metrics={'train_runtime': 18762.6769, 'train_samples_per_second': 1.599, 'train_steps_per_second': 0.05, 'total_flos': 871448892079332.0, 'train_loss': 0.15797887529645646, 'epoch': 2.0})

In [ ]:
trainer.save_model()

# **✔️ 6. Chance Level 이 넘는지 확인**

> Chance Level 0.25를 넘는 것으로 확인 되었다.



# **🔍 7. 모델 검증(Validation)**

In [20]:
from transformers import TextClassificationPipeline

pipe = TextClassificationPipeline(
    model = model,
    tokenizer = tokenizer,
    device=-1,
    return_all_scores=True,
    function_to_apply='sigmoid'
    )

In [21]:
def get_predicated_label(output_labels, min_score):
    labels = []
    for label in output_labels:
        if label['score'] > min_score:
            labels.append(1)
        else:
            labels.append(0)
    return labels

In [22]:
import tqdm
from transformers.pipelines.base import KeyDataset

predicated_labels = []

for out in tqdm.tqdm(pipe(KeyDataset(dataset['valid'], '문장'))):
    predicated_labels.append(get_predicated_label(out, 0.5))

Disabling tokenizer parallelism, we're using DataLoader multithreading already
100%|██████████| 3737/3737 [08:17<00:00,  7.51it/s]


In [23]:
from sklearn.metrics import classification_report

print(classification_report(dataset['valid']['labels'], predicated_labels))

              precision    recall  f1-score   support

           0       0.81      0.80      0.80       394
           1       0.87      0.83      0.85       334
           2       0.89      0.81      0.84       280
           3       0.85      0.80      0.82       426
           4       0.88      0.82      0.85       146
           5       0.88      0.90      0.89       260
           6       0.87      0.88      0.87       290
           7       0.86      0.09      0.16       134
           8       0.76      0.60      0.68       786
           9       0.78      0.72      0.75       935

   micro avg       0.82      0.74      0.78      3985
   macro avg       0.84      0.72      0.75      3985
weighted avg       0.82      0.74      0.76      3985
 samples avg       0.76      0.74      0.75      3985



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
